In [2]:
import numpy as np
import scipy
import pandas as pd

In [5]:
import pandas as pd
import numpy as np
import glob

# Step 1: Load all CSVs
csv_files = glob.glob('data/FixedTilt/*.csv') 



In [ ]:
dataframes = []

#for file in csv_files:
    try:
        df = pd.read_csv(file, skiprows=2)  # ← SKIPS 2 metadata rows
        dataframes.append(df)
    except pd.errors.ParserError as e:
        print(f"❌ ParserError in file: {file}")
        print(e)
    except Exception as e:
        print(f"❌ Other error in file: {file}")
        print(e)

df = pd.concat(dataframes, ignore_index=True)


In [7]:
df.head()

,Year,Month,Day,Hour,Minute,Temperature,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,...,3.9550 um,3.9600 um,3.9650 um,3.9700 um,3.9750 um,3.9800 um,3.9850 um,3.9900 um,3.9950 um,4.0000 um
0,2019,1,1,0,0,9.8,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019,1,1,1,0,9.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019,1,1,2,0,9.3,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019,1,1,3,0,9.1,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019,1,1,4,0,9.6,28.0,33.0,30.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df['Timestamp'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour']])
df.set_index('Timestamp', inplace=True)


In [9]:
# Step 3: Feature engineering
df['hour'] = df.index.hour
df['dayofyear'] = df.index.dayofyear



In [11]:
print(df.columns.tolist())


['Year', 'Month', 'Day', 'Hour', 'Minute', 'Temperature', 'Clearsky DHI', 'Clearsky DNI', 'Clearsky GHI', 'Cloud Type', 'Dew Point', 'DHI', 'DNI', 'Fill Flag', 'GHI', 'Relative Humidity', 'Solar Zenith Angle', 'Surface Albedo', 'Pressure', 'Precipitable Water', 'Wind Direction', 'Wind Speed', 'Si (BPR)', 'Si (Wacker)', 'Si (Eurosil)', 'GaAs (Bauhuis et al., 2009)', 'InGap (Gray, 2008)', 'InGap (McGambridge, 2011)', 'CdTe', 'Solar Azimuth Angle', 'Panel Tilt', 'Panel Azimuth Angle', '0.2800 um', '0.2805 um', '0.2810 um', '0.2815 um', '0.2820 um', '0.2825 um', '0.2830 um', '0.2835 um', '0.2840 um', '0.2845 um', '0.2850 um', '0.2855 um', '0.2860 um', '0.2865 um', '0.2870 um', '0.2875 um', '0.2880 um', '0.2885 um', '0.2890 um', '0.2895 um', '0.2900 um', '0.2905 um', '0.2910 um', '0.2915 um', '0.2920 um', '0.2925 um', '0.2930 um', '0.2935 um', '0.2940 um', '0.2945 um', '0.2950 um', '0.2955 um', '0.2960 um', '0.2965 um', '0.2970 um', '0.2975 um', '0.2980 um', '0.2985 um', '0.2990 um', '0.299

In [10]:
# Step 4: Select features and target
features = ['GHI', 'DHI', 'DNI', 'Temperature', 'Relative Humidity', 'hour', 'dayofyear']
target = 'GHI'

In [14]:
df.head(5)

,Year,Month,Day,Hour,Minute,Temperature,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,...,3.9650 um,3.9700 um,3.9750 um,3.9800 um,3.9850 um,3.9900 um,3.9950 um,4.0000 um,hour,dayofyear
Timestamp,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,2019,1,1,0,0,9.8,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
2019-01-01 01:00:00,2019,1,1,1,0,9.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
2019-01-01 02:00:00,2019,1,1,2,0,9.3,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1
2019-01-01 03:00:00,2019,1,1,3,0,9.1,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,1
2019-01-01 04:00:00,2019,1,1,4,0,9.6,28.0,33.0,30.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1


In [15]:
df.to_csv('Ultimate_2019_2022_FixedTilt_Data.tsv', index=False)

In [16]:
df.to_csv('Ultimate_2019_2022_FixedTilt_Data.csv', index=False)

In [17]:
df.head()  # Display the first few rows of the DataFrame to verify the changes

,Year,Month,Day,Hour,Minute,Temperature,Clearsky DHI,Clearsky DNI,Clearsky GHI,Cloud Type,...,3.9650 um,3.9700 um,3.9750 um,3.9800 um,3.9850 um,3.9900 um,3.9950 um,4.0000 um,hour,dayofyear
Timestamp,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,2019,1,1,0,0,9.8,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
2019-01-01 01:00:00,2019,1,1,1,0,9.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
2019-01-01 02:00:00,2019,1,1,2,0,9.3,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1
2019-01-01 03:00:00,2019,1,1,3,0,9.1,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,1
2019-01-01 04:00:00,2019,1,1,4,0,9.6,28.0,33.0,30.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1


In [12]:
X, y = [], []
lookback = 72
forecast_horizon = 24

data = df[features + [target]].dropna().values

for i in range(lookback, len(data) - forecast_horizon):
    X.append(data[i - lookback:i, :-1])  # past features
    y.append(data[i:i + forecast_horizon, -1])  # future GHI

X = np.array(X)
y = np.array(y)
print(f"X shape: {X.shape}, y shape: {y.shape}")

X shape: (34968, 72, 7), y shape: (34968, 24)


In [13]:
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]
print(f"Train shapes: X={X_train.shape}, y={y_train.shape}")
print(f"Test shapes: X={X_test.shape}, y={y_test.shape}")

Train shapes: X=(27974, 72, 7), y=(27974, 24)
Test shapes: X=(6994, 72, 7), y=(6994, 24)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False),
    Dense(24)  # 24 output values (1 per future hour)
])

model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0.2)


In [ ]:
y_pred = model.predict(X_test)

from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

mse = mean_squared_error(y_test.flatten(), y_pred.flatten())
print("Test MSE:", mse)

# Plot for one random prediction
import random
idx = random.randint(0, len(y_test) - 1)

plt.figure(figsize=(10,4))
plt.plot(y_test[idx], label='Actual')
plt.plot(y_pred[idx], label='Predicted')
plt.title("GHI Forecast (Next 24 Hours)")
plt.xlabel("Hour")
plt.ylabel("GHI")
plt.legend()
plt.show()
